# Regression Week 2: Multiple Regression (Interpretation)

The goal of this first notebook is to explore multiple regression and feature engineering with existing Turi Create functions.

In this notebook you will use data on house sales in King County to predict prices using multiple regression. You will:
* Use Pandas to do some feature engineering
* Use built-in Turi Create functions to compute the regression weights (coefficients/parameters)
* Given the regression weights, predictors and outcome write a function to compute the Residual Sum of Squares
* Look at coefficients and interpret their meanings
* Evaluate multiple models via RSS

# Fire up Turi Create

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [15]:
sales = pd.read_csv('kc_house_data.csv')
sales_train = pd.read_csv('kc_house_train_data.csv')
sales_test = pd.read_csv('kc_house_test_data.csv')
sales.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,...,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2487200875,20141209T000000,604000.0,4,3.00,1960,5000,1.0,0,0,...,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,1954400510,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,...,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503


# Learning a multiple regression model

Recall we can use the following code to learn a multiple regression model predicting 'price' based on the following features:
example_features = ['sqft_living', 'bedrooms', 'bathrooms'] on training data with the following code:

(Aside: We set validation_set = None to ensure that the results are always the same)

In [17]:
train_y = sales_train['price']
train_X = sales_train.drop('price', axis = 1) 
test_y = sales_test['price']
test_X = sales_test.drop('price', axis = 1) 

In [18]:
data1_train = train_X[['sqft_living', 'bedrooms', 'bathrooms']]
data1_test = test_X[['sqft_living', 'bedrooms', 'bathrooms']]

In [22]:
data1_LReg = LinearRegression()
data1_LReg.fit(data1_train, train_y)

LinearRegression()

Now that we have fitted the model we can extract the regression weights (coefficients) as an SFrame as follows:

In [23]:
print('data 1\'s slopes is : ', data1_LReg.coef_)
print('data 1\'s intercept is : ', data1_LReg.intercept_)

data 1's slopes is :  [   315.40669062 -65081.88711588   6942.16598637]
data 1's intercept is :  87912.86581496452


# Making Predictions

In the gradient descent notebook we use numpy to do our regression. In this book we will use existing Turi Create functions to analyze multiple regressions. 

Recall that once a model is built we can use the .predict() function to find the predicted values for data we pass. For example using the example model above:

In [25]:
predictions = data1_LReg.predict(data1_train)
predictions[0] # should be 271789.505878

271789.2653799699

# Compute RSS

Now that we can make predictions given the model, let's write a function to compute the RSS of the model. Complete the function below to calculate RSS given the model, data, and the outcome.

In [26]:
def get_residual_sum_of_squares(model, data, outcome):
    # First get the predictions
    y_hat = model.predict(data)
    # Then compute the residuals/errors
    err = outcome - y_hat
    # Then square and add them up
    RSS = np.sum(err**2)
    return(RSS)    

Test your function by computing the RSS on TEST data for the example model:

In [29]:
rss_example_train = get_residual_sum_of_squares(data1_LReg, data1_test, test_y)
rss_example_train # should be 2.7376153833e+14

273761940583133.7

# Create some new features

Although we often think of multiple regression as including multiple different features (e.g. # of bedrooms, squarefeet, and # of bathrooms) but we can also consider transformations of existing features e.g. the log of the squarefeet or even "interaction" features such as the product of bedrooms and bathrooms.

You will use the logarithm function to create a new feature. so first you should import it from the math library.

In [34]:
from math import log

Next create the following 4 new features as column in both TEST and TRAIN data:
* bedrooms_squared = bedrooms\*bedrooms
* bed_bath_rooms = bedrooms\*bathrooms
* log_sqft_living = log(sqft_living)
* lat_plus_long = lat + long 
As an example here's the first one:

In [32]:
train_X['bedrooms_squared'] = train_X['bedrooms'].apply(lambda x: x**2)
test_X['bedrooms_squared'] = test_X['bedrooms'].apply(lambda x: x**2)

In [35]:
# create the remaining 3 features in both TEST and TRAIN data
train_X['bed_bath_rooms'] = train_X['bedrooms']*train_X['bathrooms']
test_X['bed_bath_rooms'] = test_X['bedrooms']*test_X['bathrooms']
train_X['log_sqft_living'] = train_X['sqft_living'].apply(lambda x: log(x))
test_X['log_sqft_living'] = test_X['sqft_living'].apply(lambda x: log(x))
train_X['lat_plus_long'] = train_X['lat']+train_X['long']
test_X['lat_plus_long'] = test_X['lat']+test_X['long']

* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this feature will mostly affect houses with many bedrooms.
* bedrooms times bathrooms gives what's called an "interaction" feature. It is large when *both* of them are large.
* Taking the log of squarefeet has the effect of bringing large values closer together and spreading out small values.
* Adding latitude to longitude is totally non-sensical but we will do it anyway (you'll see why)

**Quiz Question: What is the mean (arithmetic average) value of your 4 new features on TEST data? (round to 2 digits)**

In [37]:
round(test_X['bedrooms_squared'].mean(), 2)

12.45

In [38]:
round(test_X['bed_bath_rooms'].mean(), 2)

7.5

In [39]:
round(test_X['log_sqft_living'].mean(), 2)

7.55

In [40]:
round(test_X['lat_plus_long'].mean(), 2)

-74.65

# Learning Multiple Models

Now we will learn the weights for three (nested) models for predicting house prices. The first model will have the fewest features the second model will add one more feature and the third will add a few more:
* Model 1: squarefeet, # bedrooms, # bathrooms, latitude & longitude
* Model 2: add bedrooms\*bathrooms
* Model 3: Add log squarefeet, bedrooms squared, and the (nonsensical) latitude + longitude

In [41]:
model_1_features = ['sqft_living', 'bedrooms', 'bathrooms', 'lat', 'long']
model_2_features = ['sqft_living', 'bedrooms', 'bathrooms', 'lat', 'long', 'bed_bath_rooms']
model_3_features = ['sqft_living', 'bedrooms', 'bathrooms', 'lat', 'long',
                    'bed_bath_rooms', 'bedrooms_squared', 'log_sqft_living', 'lat_plus_long']

Now that you have the features, learn the weights for the three different models for predicting target = 'price' using turicreate.linear_regression.create() and look at the value of the weights/coefficients:

In [42]:
# Learn the three models: (don't forget to set validation_set = None)
model_1 = LinearRegression()
model_1.fit(train_X[model_1_features], train_y)
model_2 = LinearRegression()
model_2.fit(train_X[model_2_features], train_y)
model_3 = LinearRegression()
model_3.fit(train_X[model_3_features], train_y)

LinearRegression()

In [44]:
# Examine/extract each model's coefficients:
print('model 1\'s slopes is : ', model_1.coef_)
print('model 1\'s intercept is : ', model_1.intercept_)

model 1's slopes is :  [ 3.12258646e+02 -5.95865332e+04  1.57067421e+04  6.58619264e+05
 -3.09374351e+05]
model 1's intercept is :  -69075726.79256983


In [45]:
print('model 2\'s slopes is : ', model_2.coef_)
print('model 2\'s intercept is : ', model_2.intercept_)

model 2's slopes is :  [ 3.06610053e+02 -1.13446368e+05 -7.14613083e+04  6.54844630e+05
 -2.94298969e+05  2.55796520e+04]
model 2's intercept is :  -66867968.87107893


**Quiz Question: What is the sign (positive or negative) for the coefficient/weight for 'bathrooms' in model 1?**

**Quiz Question: What is the sign (positive or negative) for the coefficient/weight for 'bathrooms' in model 2?**

Think about what this means.

# Comparing multiple models

Now that you've learned three models and extracted the model weights we want to evaluate which model is best.

First use your functions from earlier to compute the RSS on TRAINING Data for each of the three models.

In [46]:
# Compute the RSS on TRAINING data for each of the three models and record the values:
model1_rss_train = get_residual_sum_of_squares(model_1, train_X[model_1_features], train_y)
model1_rss_train

967879963049545.8

In [47]:
model2_rss_train = get_residual_sum_of_squares(model_2, train_X[model_2_features], train_y)
model2_rss_train

958419635074070.9

In [48]:
model3_rss_train = get_residual_sum_of_squares(model_3, train_X[model_3_features], train_y)
model3_rss_train

903436455050478.2

**Quiz Question: Which model (1, 2 or 3) has lowest RSS on TRAINING Data?** Is this what you expected?

Now compute the RSS on on TEST data for each of the three models.

In [49]:
# Compute the RSS on TESTING data for each of the three models and record the values:
model1_rss_test = get_residual_sum_of_squares(model_1, test_X[model_1_features], test_y)
model1_rss_test

225500469795490.38

In [50]:
model2_rss_test = get_residual_sum_of_squares(model_2, test_X[model_2_features], test_y)
model2_rss_test

223377462976467.2

In [51]:
model3_rss_test = get_residual_sum_of_squares(model_3, test_X[model_3_features], test_y)
model3_rss_test

259236319207180.34

**Quiz Question: Which model (1, 2 or 3) has lowest RSS on TESTING Data?** Is this what you expected? Think about the features that were added to each model from the previous.